## IMPORT DES BIBLIOTHEQUES

In [1]:
from djitellopy import Tello
import tellopy
import cv2 as cv 
import socket, time
from time import sleep
import datetime
import os
from threading import Thread

# Utilisation  de `Tellopy`

* Décollage et attérissage 

In [2]:

# Paramètres de communication
host = ''
port = 9000
locaddr = (host,port)

#Connexion à l'adresse IP du drône pour entamer la communication 
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
tello_address = ('192.168.10.1', 8889)
sock.bind(locaddr)


sock.sendto('command'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Takeoff
sock.sendto('takeoff'.encode(encoding="utf-8"), tello_address)
time.sleep(5)


#Rotate clockwise 360
sock.sendto('tcw 360'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Land
sock.sendto('land'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Utilisation de  `Djitellopy`

*  Décollage , photo et attérissage 

In [3]:
tello= Tello () 
tello.connect() # Connexion
tello.takeoff() # Décollage 
tello.streamon() # Activation de la caméra

time.sleep(2) # Tempo de 2s 
frame_read = tello.get_frame_read() # Capture d'impage 
cv.imwrite("testnew.png", frame_read.frame) # Stockage en local 
time.sleep(2)
tello.land()   # Attérissage

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'takeoff'
[INFO] tello.py - 461 - Response takeoff: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'
[INFO] tello.py - 437 - Send command: 'land'
[INFO] tello.py - 461 - Response land: 'ok'


*  Capture de vidéo

In [2]:
tello = Tello()
tello.connect()

KeepRecording= True
tello.streamon()

frame_read = tello.get_frame_read()

def videoRecorder():
    height, width,_= frame_read.frame.shape
    video = cv.VideoWriter('Video.avi', cv.VideoWriter_fourcc(*'XVID'), 30, (width,height))
    
    while KeepRecording:
        video.write(frame_read.frame)
        time.sleep(1/30)
    video.release()
    
recorder  = Thread(target=videoRecorder)
recorder.start()

tello.takeoff()
tello.land()
KeepRecording= False
recorder.join()



[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'
[INFO] tello.py - 437 - Send command: 'takeoff'
[INFO] tello.py - 461 - Response takeoff: 'ok'
[INFO] tello.py - 437 - Send command: 'land'
[INFO] tello.py - 461 - Response land: 'ok'
Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\mouha\anaconda3\envs\envniane\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\mouha\anaconda3\envs\envniane\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mouha\anaconda3\envs\envniane\lib\site-packages\djitellopy\tello.py", line 1065, in update_frame
    self.stop()
  File "C:\Users\mouha\anaconda3\envs\envniane\lib\site-packages\djitellopy\tello.py", line 1074